### Leer todos los datos que son requeridos

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/commom_functions"

In [0]:
%run "../includes/configuration"

In [0]:
movies_df = spark.read.parquet(f"{silver_folder_path}/movies").filter(
    f"file_date = '{v_file_date}'"
)


In [0]:
country_df = spark.read.parquet(f"{silver_folder_path}/countries")

In [0]:
production_country_df = spark.read.parquet(f"{silver_folder_path}/productions_countries").filter(
    f"file_date = '{v_file_date}'"
)

In [0]:
production_company_df = spark.read.parquet(f"{silver_folder_path}/productions_companies").filter(
    f"file_date = '{v_file_date}'"
)

In [0]:
movie_company_df = spark.read.parquet(f"{silver_folder_path}/movies_companies").filter(
    f"file_date = '{v_file_date}'"
)



### JOIN entre "country" y "production_country"

In [0]:
country_prod_coun_df = country_df.join(
  production_country_df,
  production_country_df.country_id == country_df.country_id,
  "inner"
).select(
  country_df.country_name,
  production_country_df.movie_id
)


### JOIN entre "production_company" y "movie_company"

In [0]:
prod_comp_mov_comp_df = production_company_df.join(
  movie_company_df,
  movie_company_df.company_id == production_company_df.company_id,
  "inner"
).select(
  movie_company_df.movie_id,
  production_company_df.company_name
)


### JOIN entre "movies_df", "country_prod_coun_df" y "prod_comp_mov_comp_df"

In [0]:
movie_filter_df = movies_df.filter("year_release_date >= 2010")

In [0]:
results_movies_country_prod_company_df = movie_filter_df.join(
  country_prod_coun_df,
  movie_filter_df.movie_id == country_prod_coun_df.movie_id,
  "inner"
).join(
  prod_comp_mov_comp_df,
  movie_filter_df.movie_id == prod_comp_mov_comp_df.movie_id,
  "inner"
)

In [0]:
from pyspark.sql.functions import lit

In [0]:
results_df = results_movies_country_prod_company_df.select(
    "title", "budget", "revenue", "duration_time", "release_date", "country_name", "company_name" 
).withColumn(
    "created_date", lit(v_file_date)
)

In [0]:
results_order_by_df = results_df.orderBy(results_df.title.asc())


In [0]:
display(results_order_by_df)

### Escribir datos en el DataLake en formato "Parquet"

In [0]:
overwrite_partition(results_order_by_df, "movie_gold", "results_country_prod_company", "created_date")

In [0]:
#results_order_by_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_country_prod_company")

results_order_by_df.write.mode("append").partitionBy("created_date").format("parquet").saveAsTable("movie_gold.results_country_prod_company")

In [0]:
#display(spark.read.parquet(f"{gold_folder_path}/results_country_prod_company"))

In [0]:
%sql
select created_date, count(1) from movie_gold.results_country_prod_company group by created_date

In [0]:
%sql
select * from movie_gold.results_country_prod_company